Merge metadata and annotations 

In [17]:
import os
import pandas as pd

metadata_filename='..' + os.sep + 'data' + os.sep +'metadata.csv'
metadata=pd.read_csv(metadata_filename)

annotations=pd.read_excel(r"C:\Users\tettret\OneDrive - DFDS\Desktop\ITU\Data Science Project\Classifier\Queen_snakes_imageids_new.xlsx")
dots=pd.read_excel(r"C:\Users\tettret\OneDrive - DFDS\Desktop\ITU\Data Science Project\Classifier\dots_merged.xlsx")
dots['img_id_cleaned']=dots['image name'].str.split('.').str[0]
metadata['img_id_cleaned']=metadata['img_id'].str.split('.').str[0]
annotations['img_id_cleaned']=annotations['Image_id'].str.split('.').str[0]
annotations['img_id_cleaned'] = annotations['img_id_cleaned'].str.replace('_mask', '')


annotations_metadata=pd.merge(annotations,metadata[['diagnostic','img_id','img_id_cleaned']], on='img_id_cleaned', how="left")
annotations_metadata_dots=pd.merge(annotations_metadata,dots, on="img_id_cleaned", how="left")
annotations_metadata_dots

,Unnamed: 0,Image_id,Annotator ID 1,Annotator ID 2,Annotator ID 3,Asymmetry 1,Asymmetry 2,Asymmetry 3,Color 1,Color 2,...,Dots and globules 1,Dots and globules 2,Dots and globules 3,img_id_cleaned,diagnostic,img_id,image name,green circle found,dots and globules 1,dots and globules 2
0,0,PAT_1180_650_843_mask.png.jpg,q1,NaN,q4,2.0,NaN,0.0,3.0,NaN,...,0.0,NaN,0.0,PAT_1180_650_843,ACK,PAT_1180_650_843.png,PAT_1180_650_843.png.jpg,1,0,0.0
1,1,PAT_1185_679_115_mask.png.jpg,q1,NaN,q4,1.0,NaN,1.0,3.0,NaN,...,0.0,NaN,0.0,PAT_1185_679_115,SEK,PAT_1185_679_115.png,PAT_1185_679_115.png.jpg,1,1,1.0
2,2,PAT_1202_722_827_mask.png.jpg,q1,NaN,q4,2.0,NaN,1.0,2.0,NaN,...,1.0,NaN,1.0,PAT_1202_722_827,SEK,PAT_1202_722_827.png,PAT_1202_722_827.png.jpg,1,1,1.0
3,3,PAT_1260_894_63_mask.png.jpg,q1,NaN,q4,2.0,NaN,1.0,2.0,NaN,...,0.0,NaN,0.0,PAT_1260_894_63,ACK,PAT_1260_894_63.png,PAT_1260_894_63.png.jpg,1,0,0.0
4,4,PAT_1303_1077_501_mask.png.jpg,q1,NaN,q4,2.0,NaN,0.0,3.0,NaN,...,0.0,NaN,0.0,PAT_1303_1077_501,ACK,PAT_1303_1077_501.png,PAT_1303_1077_501.png.jpg,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,122,PAT_388_4500_103_mask.png.jpg,q3,q2,NaN,2.0,2.0,NaN,2.0,2.0,...,0.0,0.0,NaN,PAT_388_4500_103,ACK,PAT_388_4500_103.png,PAT_388_4500_103.png.jpg,1,0,0.0
123,123,PAT_404_805_575_mask.png.jpg,q3,q2,NaN,1.0,1.0,NaN,2.0,3.0,...,1.0,1.0,NaN,PAT_404_805_575,ACK,PAT_404_805_575.png,PAT_404_805_575.png.jpg,0,0,0.0
124,124,PAT_406_1542_754_mask.png.jpg,q3,q2,NaN,0.0,0.0,NaN,4.0,4.0,...,1.0,1.0,NaN,PAT_406_1542_754,BCC,PAT_406_1542_754.png,PAT_406_1542_754.png.jpg,1,1,1.0
125,125,PAT_406_808_560_mask.png.jpg,q3,q2,NaN,1.0,0.0,NaN,3.0,4.0,...,1.0,1.0,NaN,PAT_406_808_560,BCC,PAT_406_808_560.png,PAT_406_808_560.png.jpg,1,1,1.0


In [2]:
import os
from os.path import exists
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Import our own file that has the feature extraction functions
from extract_features import extract_features



#-------------------
# Main script
#-------------------


#Where is the raw data
file_data = '..' + os.sep + 'data' + os.sep +'metadata.csv'
path_image = '..' + os.sep + 'data' + os.sep + 'images' + os.sep + 'imgs_part_1'
path_mask = '..' + os.sep + 'data' + os.sep + 'images' + os.sep + 'Queen_snakes_masks'    
  
#Where we will store the features
file_features = 'features/features.csv'


#Read meta-data into a Pandas dataframe
df = pd.read_csv(file_data)

# Extract image IDs and labels from the data. 
#image_id = list(df['img_id'])
image_id = [id + ".jpg" for id in df['img_id']]
mask_id = [id.replace('.png', '_mask.png') + '.jpg' for id in df['img_id']]
label = np.array(df['diagnostic'])

# Here you could decide to filter the data in some way (see task 0)
# For example you can have a file selected_images.csv which stores the IDs of the files you need
is_nevus =  label == 'NEV'

num_images = len(image_id)


#Make array to store features
feature_names = ['assymetry','colours','dots and globules']
num_features = len(feature_names)
features = np.zeros([num_images,num_features], dtype=np.float16)  


#Loop through all images (now just 10 for demonstration)
for i in np.arange(num_images):
    
    # Define filenames related to this image
    file_image = path_image + os.sep + image_id[i] 
    file_image_mask = path_mask + os.sep + df['img_id'][i].replace('.png', '_mask.png') + '.jpg' 
    
    if exists(file_image):
        
        if exists(file_image_mask):
        # Read the image
            im = cv2.imread(file_image)
            mask=cv2.imread(file_image_mask, cv2.IMREAD_GRAYSCALE)
            #im = np.float16(im)  
            #mask = np.float16(mask)  
            # Measure features - this does not do anything useful yet!
            x = extract_features(im,mask) 
            
            # Store in the variable we created before
            features[i,:] = x
       
        
#Save the image_id used + features to a file   
df_features = pd.DataFrame(features, columns=feature_names)     
df_features.to_csv(file_features, index=False)  
    

process images

In [2]:
import os
from os.path import exists
import pandas as pd
import numpy as np
import cv2

# Import our own file that has the feature extraction functions
from extract_features import extract_features

#-------------------
# Main script
#-------------------

# Where is the raw data
file_data = '..' + os.sep + 'data' + os.sep + 'metadata.csv'
path_image = '..' + os.sep + 'data' + os.sep + 'images' + os.sep + 'images_evaluate'
path_mask = '..' + os.sep + 'data' + os.sep + 'images' + os.sep + 'masks_evaluate'    

# Where we will store the features
file_features = 'features/features_evaluate.csv'

# Read meta-data into a Pandas dataframe
df = pd.read_csv(file_data)

# Extract image IDs and labels from the data.
#image_id = [id + ".jpg" for id in df['img_id']]
image_id =  list(df['img_id'])
#mask_id = [id.replace('.png', '_mask.png') + '.jpg' for id in df['img_id']]
mask_id = [id.replace('.png', '_mask.png') for id in df['img_id']]
label = np.array(df['diagnostic'])

# Make array to store features and list for valid image IDs
feature_names = ['assymetry', 'colours', 'dots and globules', 'compactness']
num_features = len(feature_names)
features = []
valid_image_ids = []

# Loop through all images (limited by num_images)
num_images = len(image_id)
for i in np.arange(num_images):
    # Define filenames related to this image
    file_image = path_image + os.sep + image_id[i]
    file_image_mask = path_mask + os.sep + mask_id[i]

    # Check if both the image and mask files exist
    if exists(file_image) and exists(file_image_mask):
        # Read the image and mask
        im = cv2.imread(file_image)
        mask = cv2.imread(file_image_mask, cv2.IMREAD_GRAYSCALE)

        # Measure features
        x = extract_features(im, mask)

        # Store in the list we created before
        features.append(x)

        # Keep track of the valid image ID
        valid_image_ids.append(image_id[i])

# Create DataFrame from the features list and add image IDs
df_features = pd.DataFrame(features, columns=feature_names)
df_features['image_id'] = valid_image_ids

# Save the image_id used + features to a file
#df_features.to_excel(file_features, index=False)
df_features.to_csv(file_features, index=False)


train_classifiers

In [3]:
import os
import pandas as pd
import numpy as np

# Default packages for the minimum example
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score #example for measuring performance


import pickle #for saving/loading trained classifiers


#Where are the files
file_data = '..' + os.sep + 'data' + os.sep +'metadata.csv'
df = pd.read_csv(file_data)
label = np.array(df['diagnostic'])


#Where did we store the features?
file_features = 'features/features.csv'
feature_names = ['assymetry', 'colours', 'dots and globules', 'compactness']

# Load the features - remember the example features are not informative
df_features = pd.read_csv(file_features)


# Make the dataset, you can select different classes (see task 0)
x = np.array(df_features[feature_names])
y =  label == 'NEV'   #now True means healthy nevus, False means something else
patient_id = df['patient_id']


#Prepare cross-validation - images from the same patient must always stay together
num_folds = 5
group_kfold = GroupKFold(n_splits=num_folds)
group_kfold.get_n_splits(x, y, patient_id)


#Different classifiers to test out
classifiers = [
    KNeighborsClassifier(1),
    KNeighborsClassifier(5)
]
num_classifiers = len(classifiers)

      
acc_val = np.empty([num_folds,num_classifiers])

for i, (train_index, val_index) in enumerate(group_kfold.split(x, y, patient_id)):
    
    x_train = x[train_index,:]
    y_train = y[train_index]
    x_val = x[val_index,:]
    y_val = y[val_index]
    
    
    for j, clf in enumerate(classifiers): 
        
        #Train the classifier
        clf.fit(x_train,y_train)
    
        #Evaluate your metric of choice (accuracy is probably not the best choice)
        acc_val[i,j] = accuracy_score(y_val, clf.predict(x_val))
   
    
#Average over all folds
average_acc = np.mean(acc_val,axis=0) 
   
print('Classifier 1 average accuracy={:.3f} '.format(average_acc[0]))
print('Classifier 2 average accuracy={:.3f} '.format(average_acc[1]))



#Let's say you now decided to use the 5-NN 
classifier = KNeighborsClassifier(n_neighbors = 5)

#It will be tested on external data, so we can try to maximize the use of our available data by training on 
#ALL of x and y
classifier = classifier.fit(x,y)

#This is the classifier you need to save using pickle, add this to your zip file submission
filename = 'groupXY_classifier.sav'
pickle.dump(classifier, open(filename, 'wb'))

ValueError: Found input variables with inconsistent numbers of samples: [127, 2298, 2298]

In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score
import pickle

# Load the metadata and features data
metadata_path =  '..' + os.sep + 'data' + os.sep +'metadata.csv'
features_path = 'features/features.csv'
metadata_df = pd.read_csv(metadata_path)
features_df = pd.read_csv(features_path)
feature_names = ['colours', 'dots and globules', 'compactness']

# Correct the 'image_id' in features to match 'img_id' in metadata
features_df['image_id'] = features_df['image_id'].str.replace('.png.jpg', '.png')

# Merge features with metadata on 'image_id'/'img_id'
combined_df = features_df.merge(metadata_df[['img_id', 'diagnostic','patient_id']], left_on='image_id', right_on='img_id', how='left')

# Prepare the dataset
feature_columns = combined_df.columns[:-2]  # Exclude 'image_id' and 'img_id' columns
X = combined_df[feature_names].to_numpy()
y = combined_df['diagnostic'] == 'NEV'  # True for 'NEV', False otherwise
patient_id = combined_df['patient_id']

# Prepare cross-validation
num_folds = 5
group_kfold = GroupKFold(n_splits=num_folds)
group_kfold.get_n_splits(X, y, patient_id)

# Initialize classifiers
classifiers = [
    KNeighborsClassifier(1),
    KNeighborsClassifier(5)
]

# Initialize accuracy storage
acc_val = np.empty([num_folds, len(classifiers)])

# Perform cross-validation
for i, (train_index, val_index) in enumerate(group_kfold.split(X, y, patient_id)):
    x_train, y_train = X[train_index], y[train_index]
    x_val, y_val = X[val_index], y[val_index]
    
    for j, clf in enumerate(classifiers):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_val)
        acc_val[i, j] = accuracy_score(y_val, y_pred)

# Calculate average accuracy
average_acc = np.mean(acc_val, axis=0)
print(f'Classifier 1 average accuracy={average_acc[0]:.3f}')
print(f'Classifier 2 average accuracy={average_acc[1]:.3f}')

# Select the classifier and train on all data
best_classifier = KNeighborsClassifier(n_neighbors=5)
best_classifier.fit(X, y)

# Save the classifier
filename = 'groupXY_classifier.sav'
pickle.dump(best_classifier, open(filename, 'wb'))

# Correct file paths if needed when running this script in your local environment.


Classifier 1 average accuracy=0.810
Classifier 2 average accuracy=0.897


more classifiers

In [12]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import pickle

# Load the metadata and features data
metadata_path = '..' + os.sep + 'data' + os.sep + 'metadata.csv'
features_path = 'features/features.csv'
metadata_df = pd.read_csv(metadata_path)
features_df = pd.read_csv(features_path)
feature_names = ['assymetry', 'colours', 'dots and globules', 'compactness']

# Correct the 'image_id' in features to match 'img_id' in metadata
features_df['image_id'] = features_df['image_id'].str.replace('.png.jpg', '.png')

# Merge features with metadata on 'image_id'/'img_id'
combined_df = features_df.merge(metadata_df[['img_id', 'diagnostic', 'patient_id']], left_on='image_id', right_on='img_id', how='left')

# Prepare the dataset
X = combined_df[feature_names].to_numpy()
y = combined_df['diagnostic'] == 'NEV'
patient_id = combined_df['patient_id']

# Prepare cross-validation
num_folds = 5
group_kfold = GroupKFold(n_splits=num_folds)
group_kfold.get_n_splits(X, y, patient_id)

# Initialize classifiers
classifiers = [
    KNeighborsClassifier(1),
    KNeighborsClassifier(5),
    make_pipeline(StandardScaler(), SVC(probability=True)),
    RandomForestClassifier(n_estimators=100, random_state=42),
    GradientBoostingClassifier(n_estimators=100, random_state=42),
    AdaBoostClassifier(n_estimators=100, random_state=42),
    DecisionTreeClassifier(random_state=42),
    LogisticRegression(max_iter=1000),
    SGDClassifier(max_iter=1000, tol=1e-3),
    GaussianNB(),
    MLPClassifier(max_iter=1000)
]

# Initialize accuracy storage
acc_val = np.empty((num_folds, len(classifiers)))

# Perform cross-validation
for i, (train_index, val_index) in enumerate(group_kfold.split(X, y, patient_id)):
    x_train, y_train = X[train_index], y[train_index]
    x_val, y_val = X[val_index], y[val_index]
    
    for j, clf in enumerate(classifiers):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_val)
        acc_val[i, j] = accuracy_score(y_val, y_pred)

# Calculate average accuracy for each classifier
average_acc = np.mean(acc_val, axis=0)
for idx, clf in enumerate(classifiers):
    print(f'Classifier {idx + 1} ({clf.__class__.__name__}): average accuracy={average_acc[idx]:.3f}')

# Select the best classifier and train on all data
best_index = np.argmax(average_acc)
best_classifier = classifiers[best_index]
best_classifier.fit(X, y)

# Save the best classifier
filename = 'best_classifier.sav'
pickle.dump(best_classifier, open(filename, 'wb'))


c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent

Classifier 1 (KNeighborsClassifier): average accuracy=0.819
Classifier 2 (KNeighborsClassifier): average accuracy=0.889
Classifier 3 (Pipeline): average accuracy=0.897
Classifier 4 (RandomForestClassifier): average accuracy=0.866
Classifier 5 (GradientBoostingClassifier): average accuracy=0.866
Classifier 6 (AdaBoostClassifier): average accuracy=0.889
Classifier 7 (DecisionTreeClassifier): average accuracy=0.819
Classifier 8 (LogisticRegression): average accuracy=0.897
Classifier 9 (SGDClassifier): average accuracy=0.828
Classifier 10 (GaussianNB): average accuracy=0.842
Classifier 11 (MLPClassifier): average accuracy=0.897


c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Saving classifiers

In [8]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import pickle

# Load the metadata and features data
metadata_path = '..' + os.sep + 'data' + os.sep + 'metadata.csv'
features_path = 'features' + os.sep + 'features.csv'
metadata_df = pd.read_csv(metadata_path)
features_df = pd.read_csv(features_path)
feature_names = ['assymetry', 'colours', 'dots and globules', 'compactness']

# Correct the 'image_id' in features to match 'img_id' in metadata
features_df['image_id'] = features_df['image_id'].str.replace('.png.jpg', '.png', regex=False)

# Merge features with metadata on 'image_id'/'img_id'
combined_df = features_df.merge(metadata_df[['img_id', 'diagnostic', 'patient_id']], left_on='image_id', right_on='img_id', how='left')

# Check for any NaNs after the merge and handle them before proceeding.
if combined_df.isnull().values.any():
    raise ValueError("NaN values detected after merge! Check the data integrity.")

# Prepare the dataset
X = combined_df[feature_names].to_numpy()
y = combined_df['diagnostic'].values == 'NEV'  # NEV is assumed to be the healthy class
patient_id = combined_df['patient_id'].values

# Prepare cross-validation
num_folds = 5
group_kfold = GroupKFold(n_splits=num_folds)

# Initialize classifiers
classifiers = [
    KNeighborsClassifier(1),
    KNeighborsClassifier(5),
    make_pipeline(StandardScaler(), SVC(probability=True)),
    RandomForestClassifier(n_estimators=100, random_state=42),
    GradientBoostingClassifier(n_estimators=100, random_state=42),
    AdaBoostClassifier(n_estimators=100, random_state=42),
    DecisionTreeClassifier(random_state=42),
    make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000)),
    make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3)),
    GaussianNB(),
    MLPClassifier(max_iter=1000)
]

# Initialize accuracy storage
acc_val = np.empty((num_folds, len(classifiers)))
classifier_names = []

# Perform cross-validation
for j, clf in enumerate(classifiers):
    classifier_name = (clf.named_steps['svc'].__class__.__name__ if 'pipeline' in str(clf)
                       else clf.__class__.__name__)
    classifier_names.append(classifier_name)
    fold_accuracies = []
    
    for i, (train_index, val_index) in enumerate(group_kfold.split(X, y, patient_id)):
        x_train, y_train = X[train_index], y[train_index]
        x_val, y_val = X[val_index], y[val_index]
        
        # Fit and predict
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_val)
        acc = accuracy_score(y_val, y_pred)
        fold_accuracies.append(acc)
        
        # Save the classifier after training on this fold
        fold_filename = f'classifier_{j}_fold_{i}.sav'
        pickle.dump(clf, open(fold_filename, 'wb'))
    
    acc_val[:, j] = fold_accuracies

# Calculate average accuracy for each classifier
average_acc = np.mean(acc_val, axis=0)
for idx, acc in enumerate(average_acc):
    print(f'Classifier {idx + 1} ({classifier_names[idx]}): average accuracy={acc:.3f}')

# Save and evaluate each classifier on the full dataset
eval_results = {}
for idx, clf in enumerate(classifiers):
    classifier_name = classifier_names[idx]
    classifier_filename = f'classifier_{idx}.sav'
    
    # Save the classifier
    pickle.dump(clf, open(classifier_filename, 'wb'))
    
    # Load the classifier
    loaded_clf = pickle.load(open(classifier_filename, 'rb'))
    
    # Predict on the full dataset and calculate evaluation metrics
    y_pred = loaded_clf.predict(X)
    acc = accuracy_score(y, y_pred)
    clf_report = classification_report(y, y_pred)
    
    eval_results[classifier_name] = {'accuracy': acc, 'report': clf_report}

# Display evaluation results
for clf_name, results in eval_results.items():
    print(f"Results for {clf_name}:")
    print(f"Accuracy: {results['accuracy']}")
    print(f"Classification Report:\n{results['report']}\n")



c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent

Classifier 1 (KNeighborsClassifier): average accuracy=0.819
Classifier 2 (KNeighborsClassifier): average accuracy=0.889
Classifier 3 (Pipeline): average accuracy=0.897
Classifier 4 (RandomForestClassifier): average accuracy=0.866
Classifier 5 (GradientBoostingClassifier): average accuracy=0.866
Classifier 6 (AdaBoostClassifier): average accuracy=0.889
Classifier 7 (DecisionTreeClassifier): average accuracy=0.819
Classifier 8 (Pipeline): average accuracy=0.897
Classifier 9 (Pipeline): average accuracy=0.780
Classifier 10 (GaussianNB): average accuracy=0.842
Classifier 11 (MLPClassifier): average accuracy=0.897


c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\User

Results for KNeighborsClassifier:
Accuracy: 0.8976377952755905
Classification Report:
              precision    recall  f1-score   support

       False       0.90      1.00      0.95       114
        True       0.00      0.00      0.00        13

    accuracy                           0.90       127
   macro avg       0.45      0.50      0.47       127
weighted avg       0.81      0.90      0.85       127


Results for Pipeline:
Accuracy: 0.8976377952755905
Classification Report:
              precision    recall  f1-score   support

       False       0.90      1.00      0.95       114
        True       0.00      0.00      0.00        13

    accuracy                           0.90       127
   macro avg       0.45      0.50      0.47       127
weighted avg       0.81      0.90      0.85       127


Results for RandomForestClassifier:
Accuracy: 0.968503937007874
Classification Report:
              precision    recall  f1-score   support

       False       0.97      1.00      0.9

c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\User

In [18]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import pickle

# Load the metadata and features data
metadata_path = '..' + os.sep + 'data' + os.sep + 'metadata.csv'
features_path = 'features' + os.sep + 'features.csv'
metadata_df = pd.read_csv(metadata_path)
features_df = pd.read_csv(features_path)
feature_names = ['assymetry', 'colours', 'dots and globules', 'compactness']

# Correct the 'image_id' in features to match 'img_id' in metadata
features_df['image_id'] = features_df['image_id'].str.replace('.png.jpg', '.png', regex=False)

# Merge features with metadata on 'image_id'/'img_id'
combined_df = features_df.merge(metadata_df[['img_id', 'diagnostic', 'patient_id']], left_on='image_id', right_on='img_id', how='left')

# Check for any NaNs after the merge and handle them before proceeding.
if combined_df.isnull().values.any():
    raise ValueError("NaN values detected after merge! Check the data integrity.")

# Prepare the dataset
X = combined_df[feature_names].to_numpy()
y = combined_df['diagnostic'].values == 'NEV'
patient_id = combined_df['patient_id'].values

# Prepare cross-validation
num_folds = 5
group_kfold = GroupKFold(n_splits=num_folds)

# Initialize classifiers
classifiers = [
    KNeighborsClassifier(1),
    KNeighborsClassifier(5),
    make_pipeline(StandardScaler(), SVC(probability=True)),
    RandomForestClassifier(n_estimators=100, random_state=42),
    GradientBoostingClassifier(n_estimators=100, random_state=42),
    AdaBoostClassifier(n_estimators=100, random_state=42),
    DecisionTreeClassifier(random_state=42),
    make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000)),
    make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3)),
    GaussianNB(),
    MLPClassifier(max_iter=1000)
]

# Initialize accuracy storage
acc_val = np.empty((num_folds, len(classifiers)))

# Perform cross-validation
for i, (train_index, val_index) in enumerate(group_kfold.split(X, y, patient_id)):
    x_train, y_train = X[train_index], y[train_index]
    x_val, y_val = X[val_index], y[val_index]
    
    for j, clf in enumerate(classifiers):
        # Fit and predict
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_val)
        acc_val[i, j] = accuracy_score(y_val, y_pred)

# Calculate average accuracy for each classifier
average_acc = np.mean(acc_val, axis=0)
for idx, clf in enumerate(classifiers):
    # Print the classifier number, name, and accuracy
    classifier_name = clf.named_steps['svc'].__class__.__name__ if 'pipeline' in str(clf) else clf.__class__.__name__
    print(f'Classifier {idx + 1} ({classifier_name}): average accuracy={average_acc[idx]:.3f}')

# Select the best classifier and train on all data
best_index = np.argmax(average_acc)
best_classifier = classifiers[best_index]
best_classifier.fit(X, y)

# Save the best classifier
filename = 'best_classifier.sav'
pickle.dump(best_classifier, open(filename, 'wb'))


c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent

Classifier 1 (KNeighborsClassifier): average accuracy=0.819
Classifier 2 (KNeighborsClassifier): average accuracy=0.889
Classifier 3 (Pipeline): average accuracy=0.897
Classifier 4 (RandomForestClassifier): average accuracy=0.866
Classifier 5 (GradientBoostingClassifier): average accuracy=0.866
Classifier 6 (AdaBoostClassifier): average accuracy=0.889
Classifier 7 (DecisionTreeClassifier): average accuracy=0.819
Classifier 8 (Pipeline): average accuracy=0.897
Classifier 9 (Pipeline): average accuracy=0.850
Classifier 10 (GaussianNB): average accuracy=0.842
Classifier 11 (MLPClassifier): average accuracy=0.897


c:\Users\tettret\Downloads\Anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


evaluate classifier

In [5]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the metadata and features data
metadata_path = '..' + os.sep + 'data' + os.sep + 'metadata.csv'
features_path = 'features/features_evaluate.csv'
metadata_df = pd.read_csv(metadata_path)
features_df = pd.read_csv(features_path)
feature_names = ['assymetry', 'colours', 'dots and globules', 'compactness']

# Correct the 'image_id' in features to match 'img_id' in metadata
#features_df['image_id'] = features_df['image_id'].str.replace('.png.jpg', '.png')

# Merge features with metadata on 'image_id'/'img_id'
combined_df = features_df.merge(metadata_df[['img_id', 'diagnostic', 'patient_id']], left_on='image_id', right_on='img_id', how='left')

# Prepare the dataset
X = combined_df[feature_names].to_numpy()
y = combined_df['diagnostic'] == 'NEV'  # NEV is assumed to be the healthy class
patient_id = combined_df['patient_id']

# Load the trained classifier
model_filename = 'best_classifier.sav'
try:
    with open(model_filename, 'rb') as model_file:
        classifier = pickle.load(model_file)
except Exception as e:
    print(f"An error occurred while loading the model: {e}")
    classifier = None

# Function to classify and evaluate
def classify_and_evaluate(X, y):
    if classifier is None:
        print("Classifier not loaded. Cannot perform classification.")
        return None, None

    # Predict the labels
    pred_labels = classifier.predict(X)

    # Calculate accuracy and confusion matrix
    accuracy = accuracy_score(y, pred_labels)
    cm = confusion_matrix(y, pred_labels)

    return accuracy, cm

# Evaluate the classifier
accuracy, cm = classify_and_evaluate(X, y)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", cm)



Accuracy: 0.8861788617886179
Confusion Matrix:
 [[109   0]
 [ 14   0]]


In [6]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the metadata and features data
metadata_path = '..' + os.sep + 'data' + os.sep + 'metadata.csv'
features_path = 'features/features_evaluate.csv'
metadata_df = pd.read_csv(metadata_path)
features_df = pd.read_csv(features_path)
feature_names = ['assymetry', 'colours', 'dots and globules', 'compactness']

# Merge features with metadata on 'image_id'/'img_id'
combined_df = features_df.merge(metadata_df[['img_id', 'diagnostic', 'patient_id']], left_on='image_id', right_on='img_id', how='left')

# Prepare the dataset
X = combined_df[feature_names].to_numpy()
y = combined_df['diagnostic'] == 'NEV'  # NEV is assumed to be the healthy class
patient_id = combined_df['patient_id']

# Function to load a classifier and evaluate it
def load_and_evaluate(model_filename, X, y):
    try:
        with open(model_filename, 'rb') as model_file:
            classifier = pickle.load(model_file)
        print(f"Loaded classifier from {model_filename}")
    except Exception as e:
        print(f"An error occurred while loading the model from {model_filename}: {e}")
        return None, None

    # Predict the labels and calculate accuracy and confusion matrix
    pred_labels = classifier.predict(X)
    accuracy = accuracy_score(y, pred_labels)
    cm = confusion_matrix(y, pred_labels)
    
    return accuracy, cm

# List of classifier filenames
classifier_filenames = [f'classifier_{i}.sav' for i in range(len(os.listdir(r"C:\Users\tettret\OneDrive - DFDS\Desktop\ITU\Data Science Project\Project_data_science_queen_snakes-3\Classifier\fyp2024")))]

# Evaluate all classifiers
for model_filename in classifier_filenames:
    accuracy, cm = load_and_evaluate(model_filename, X, y)
    print(f"Results for {model_filename}:")
    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", cm)


0      False
1      False
2      False
3      False
4      False
       ...  
118     True
119    False
120    False
121    False
122    False
Name: diagnostic, Length: 123, dtype: bool

In [ ]:
import pickle

# Assuming extract_features.py is in the same directory and has a function named extract_features
from extract_features import extract_features

# The function to classify new images. The image and mask are assumed to be loaded.
def classify(img, mask):
    # Resize the image etc, if you did that during training.
    # ...

    # Extract features (the same ones that you used for training).
    # Ensure that the feature array is 2D: 1 row of features for 1 example.
    x = extract_features(img, mask).reshape(1, -1)

    # Load the trained classifier.
    # Make sure the file name matches the one you used when saving the model.
    classifier = pickle.load(open('best_classifier.sav', 'rb'))

    # Use it on this example to predict the label AND posterior probability.
    pred_label = classifier.predict(x)
    pred_prob = classifier.predict_proba(x)

    # Uncomment below to print the results if needed.
    # print('Predicted label is:', pred_label)
    # print('Predicted probability is:', pred_prob)
    return pred_label, pred_prob

In [17]:
y

array([ True,  True,  True,  True, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
        True, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [14]:
dots['img_id_cleaned']=dots['image name'].str.split('.').str[0]
dots

,image name,green circle found,dots and globules 1,dots and globules 2,dots and globules 3,img_id_cleaned
0,PAT_101_1041_898.png.jpg,1,1.0,1.0,NaN,PAT_101_1041_898
1,PAT_1063_271_448.png.jpg,0,0.0,NaN,0.0,PAT_1063_271_448
2,PAT_106_158_270.png.jpg,1,0.0,NaN,0.0,PAT_106_158_270
3,PAT_1074_322_864.png.jpg,0,0.0,NaN,0.0,PAT_1074_322_864
4,PAT_1089_375_60.png.jpg,0,0.0,NaN,0.0,PAT_1089_375_60
...,...,...,...,...,...,...
122,PAT_93_361_467.png.jpg,0,1.0,1.0,NaN,PAT_93_361_467
123,PAT_943_1793_256.png.jpg,1,1.0,1.0,NaN,PAT_943_1793_256
124,PAT_980_1849_295.png.jpg,1,1.0,1.0,NaN,PAT_980_1849_295
125,PAT_983_1854_274.png.jpg,1,1.0,1.0,NaN,PAT_983_1854_274


In [13]:
annotations_metadata

,Unnamed: 0,Image_id,Annotator ID 1,Annotator ID 2,Annotator ID 3,Asymmetry 1,Asymmetry 2,Asymmetry 3,Color 1,Color 2,Color 3,Dots and globules 1,Dots and globules 2,Dots and globules 3,img_id_cleaned,diagnostic,img_id
0,0,PAT_1180_650_843_mask.png.jpg,q1,NaN,q4,2.0,NaN,0.0,3.0,NaN,1.0,0.0,NaN,0.0,PAT_1180_650_843,ACK,PAT_1180_650_843.png
1,1,PAT_1185_679_115_mask.png.jpg,q1,NaN,q4,1.0,NaN,1.0,3.0,NaN,2.0,0.0,NaN,0.0,PAT_1185_679_115,SEK,PAT_1185_679_115.png
2,2,PAT_1202_722_827_mask.png.jpg,q1,NaN,q4,2.0,NaN,1.0,2.0,NaN,2.0,1.0,NaN,1.0,PAT_1202_722_827,SEK,PAT_1202_722_827.png
3,3,PAT_1260_894_63_mask.png.jpg,q1,NaN,q4,2.0,NaN,1.0,2.0,NaN,2.0,0.0,NaN,0.0,PAT_1260_894_63,ACK,PAT_1260_894_63.png
4,4,PAT_1303_1077_501_mask.png.jpg,q1,NaN,q4,2.0,NaN,0.0,3.0,NaN,3.0,0.0,NaN,0.0,PAT_1303_1077_501,ACK,PAT_1303_1077_501.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,122,PAT_388_4500_103_mask.png.jpg,q3,q2,NaN,2.0,2.0,NaN,2.0,2.0,NaN,0.0,0.0,NaN,PAT_388_4500_103,ACK,PAT_388_4500_103.png
123,123,PAT_404_805_575_mask.png.jpg,q3,q2,NaN,1.0,1.0,NaN,2.0,3.0,NaN,1.0,1.0,NaN,PAT_404_805_575,ACK,PAT_404_805_575.png
124,124,PAT_406_1542_754_mask.png.jpg,q3,q2,NaN,0.0,0.0,NaN,4.0,4.0,NaN,1.0,1.0,NaN,PAT_406_1542_754,BCC,PAT_406_1542_754.png
125,125,PAT_406_808_560_mask.png.jpg,q3,q2,NaN,1.0,0.0,NaN,3.0,4.0,NaN,1.0,1.0,NaN,PAT_406_808_560,BCC,PAT_406_808_560.png


In [16]:
annotations_metadata_dots=pd.merge(annotations_metadata,dots, on="img_id_cleaned", how="left")
annotations_metadata_dots

,Unnamed: 0,Image_id,Annotator ID 1,Annotator ID 2,Annotator ID 3,Asymmetry 1,Asymmetry 2,Asymmetry 3,Color 1,Color 2,...,Dots and globules 2,Dots and globules 3,img_id_cleaned,diagnostic,img_id,image name,green circle found,dots and globules 1,dots and globules 2,dots and globules 3
0,0,PAT_1180_650_843_mask.png.jpg,q1,NaN,q4,2.0,NaN,0.0,3.0,NaN,...,NaN,0.0,PAT_1180_650_843,ACK,PAT_1180_650_843.png,PAT_1180_650_843.png.jpg,1,0.0,NaN,0.0
1,1,PAT_1185_679_115_mask.png.jpg,q1,NaN,q4,1.0,NaN,1.0,3.0,NaN,...,NaN,0.0,PAT_1185_679_115,SEK,PAT_1185_679_115.png,PAT_1185_679_115.png.jpg,1,1.0,NaN,1.0
2,2,PAT_1202_722_827_mask.png.jpg,q1,NaN,q4,2.0,NaN,1.0,2.0,NaN,...,NaN,1.0,PAT_1202_722_827,SEK,PAT_1202_722_827.png,PAT_1202_722_827.png.jpg,1,1.0,NaN,1.0
3,3,PAT_1260_894_63_mask.png.jpg,q1,NaN,q4,2.0,NaN,1.0,2.0,NaN,...,NaN,0.0,PAT_1260_894_63,ACK,PAT_1260_894_63.png,PAT_1260_894_63.png.jpg,1,0.0,NaN,0.0
4,4,PAT_1303_1077_501_mask.png.jpg,q1,NaN,q4,2.0,NaN,0.0,3.0,NaN,...,NaN,0.0,PAT_1303_1077_501,ACK,PAT_1303_1077_501.png,PAT_1303_1077_501.png.jpg,0,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,122,PAT_388_4500_103_mask.png.jpg,q3,q2,NaN,2.0,2.0,NaN,2.0,2.0,...,0.0,NaN,PAT_388_4500_103,ACK,PAT_388_4500_103.png,PAT_388_4500_103.png.jpg,1,0.0,0.0,NaN
123,123,PAT_404_805_575_mask.png.jpg,q3,q2,NaN,1.0,1.0,NaN,2.0,3.0,...,1.0,NaN,PAT_404_805_575,ACK,PAT_404_805_575.png,PAT_404_805_575.png.jpg,0,0.0,0.0,NaN
124,124,PAT_406_1542_754_mask.png.jpg,q3,q2,NaN,0.0,0.0,NaN,4.0,4.0,...,1.0,NaN,PAT_406_1542_754,BCC,PAT_406_1542_754.png,PAT_406_1542_754.png.jpg,1,1.0,1.0,NaN
125,125,PAT_406_808_560_mask.png.jpg,q3,q2,NaN,1.0,0.0,NaN,3.0,4.0,...,1.0,NaN,PAT_406_808_560,BCC,PAT_406_808_560.png,PAT_406_808_560.png.jpg,1,1.0,1.0,NaN


In [5]:
annotations_metadata

,Unnamed: 0,Image_id,Annotator ID 1,Annotator ID 2,Annotator ID 3,Asymmetry 1,Asymmetry 2,Asymmetry 3,Color 1,Color 2,Color 3,Dots and globules 1,Dots and globules 2,Dots and globules 3,img_id_cleaned,diagnostic,img_id
0,0,PAT_1180_650_843_mask.png.jpg,q1,NaN,q4,2.0,NaN,0.0,3.0,NaN,1.0,0.0,NaN,0.0,PAT_1180_650_843_mask,NaN,NaN
1,1,PAT_1185_679_115_mask.png.jpg,q1,NaN,q4,1.0,NaN,1.0,3.0,NaN,2.0,0.0,NaN,0.0,PAT_1185_679_115_mask,NaN,NaN
2,2,PAT_1202_722_827_mask.png.jpg,q1,NaN,q4,2.0,NaN,1.0,2.0,NaN,2.0,1.0,NaN,1.0,PAT_1202_722_827_mask,NaN,NaN
3,3,PAT_1260_894_63_mask.png.jpg,q1,NaN,q4,2.0,NaN,1.0,2.0,NaN,2.0,0.0,NaN,0.0,PAT_1260_894_63_mask,NaN,NaN
4,4,PAT_1303_1077_501_mask.png.jpg,q1,NaN,q4,2.0,NaN,0.0,3.0,NaN,3.0,0.0,NaN,0.0,PAT_1303_1077_501_mask,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,122,PAT_388_4500_103_mask.png.jpg,q3,q2,NaN,2.0,2.0,NaN,2.0,2.0,NaN,0.0,0.0,NaN,PAT_388_4500_103_mask,NaN,NaN
123,123,PAT_404_805_575_mask.png.jpg,q3,q2,NaN,1.0,1.0,NaN,2.0,3.0,NaN,1.0,1.0,NaN,PAT_404_805_575_mask,NaN,NaN
124,124,PAT_406_1542_754_mask.png.jpg,q3,q2,NaN,0.0,0.0,NaN,4.0,4.0,NaN,1.0,1.0,NaN,PAT_406_1542_754_mask,NaN,NaN
125,125,PAT_406_808_560_mask.png.jpg,q3,q2,NaN,1.0,0.0,NaN,3.0,4.0,NaN,1.0,1.0,NaN,PAT_406_808_560_mask,NaN,NaN


In [2]:
import os
import pandas as pd
metadata_filename='..' + os.sep + 'data' + os.sep +'metadata.csv'
metadata=pd.read_csv(metadata_filename)

In [9]:
metadata.columns

Index(['patient_id', 'lesion_id', 'smoke', 'drink', 'background_father',
       'background_mother', 'age', 'pesticide', 'gender',
       'skin_cancer_history', 'cancer_history', 'has_piped_water',
       'has_sewage_system', 'fitspatrick', 'region', 'diameter_1',
       'diameter_2', 'diagnostic', 'itch', 'grew', 'hurt', 'changed', 'bleed',
       'elevation', 'img_id', 'biopsed'],
      dtype='object')